In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import math as mt

df = pd.ExcelFile('C:\\Users\\Kevin\\Documents\\MATLAB\\TESIS DENME MI TITULO\\AntiTb.xlsx')

df1 = df.parse('AntiTb_wABP')
seq = df1[['Secuencia']]

alphabet = 'ARNDCQEGHILKMFPSTWYV'
def onehot(seq_list,alphabet):
    length_seq = []
    for i in range(0,len(seq)):
        length_seq = length_seq + [len(seq.loc[i][0])]
    max_len = max(length_seq)
    onehot_seq = []
    for i in range(0,len(seq_list)):
        seq_str = seq_list.loc[i][0]
        seq_str = seq_str + '-'*(max_len - len(seq_str))
        onehot_matrix = np.zeros((len(alphabet),max_len), dtype=int)
        for j in range(0,len(seq_str)):
            ind = alphabet.find(seq_str[j])
            if ind!=-1:
                onehot_matrix[ind,j] = 1 
        onehot_matrix = onehot_matrix.transpose()
        onehot_list = []
        for k in range(0,len(seq_str)):
            for j in range(0,len(alphabet)):
                onehot_list = onehot_list + [onehot_matrix[k,j]]
        onehot_seq.append(onehot_list)
    return onehot_seq

def ordinal(seq_list,alphabet):
    length_seq = []
    for i in range(0,len(seq_list)):
        length_seq = length_seq + [len(seq_list[i])]
    max_len = max(length_seq)
    ordinal_seq = []
    for i in range(0,len(seq_list)):
        seq_str = seq_list[i]
        seq_str = seq_str + '-'*(max_len - len(seq_str))
        ordinal_vec = []
        for j in range(0,len(seq_str)):
            ind = alphabet.find(seq_str[j])
            if ind!=-1:
                ordinal_vec = ordinal_vec + [ind+1]
            else:
                ordinal_vec = ordinal_vec + [0]
        ordinal_seq.append(ordinal_vec)
    return ordinal_seq
                


def composition(seq_list,alphabet):
    AAC_list = []
    if type(seq_list)==str:
        n=1
    else:
        n=len(seq_list)
    for i in range(0,n):
        AAC_vec = [0]*len(alphabet)
        if type(seq_list)==str:
            seq_str = seq_list
        else:
            seq_str = seq_list[i]
        for j in range(0,len(seq_str)):
            ind = alphabet.find(seq_str[j])
            if ind!=-1:
                AAC_vec[ind] = AAC_vec[ind]+1
        N = len(seq_str)
        AAC_vec = [AAC_vec[i]/N for i in range(0,len(AAC_vec))]
        if type(seq_list)==str:
            for k in range(0,len(AAC_vec)):
                AAC_list = AAC_list + [AAC_vec[k]]
        else:
            AAC_list.append(AAC_vec)
    return AAC_list


def transition_pairs(seq_str,alphabet,vectorformat):
    pair_distr = np.zeros((len(alphabet),len(alphabet)),dtype=int)   ## rows=first aa, columns=second aa
    for i in range(0,len(seq_str)-1):
        if seq_str[i]!=seq_str[i+1]:
            ind_first = alphabet.find(seq_str[i])
            ind_second = alphabet.find(seq_str[i+1])
            if ind_first!=-1 and ind_second!=-1:
                pair_distr[ind_first,ind_second] = pair_distr[ind_first,ind_second] + 1
    N = len(seq_str)
    T = pair_distr
    for j in range(0,len(alphabet)):
        for i in range(j+1,len(alphabet)):
            T[i,j] = T[i,j]+T[j,i]
            T[j,i] = 0
    T = T/(N-1)
    if vectorformat:
        Tvec = []
        for j in range(0,len(alphabet)):
            for i in range(j+1,len(alphabet)):
                Tvec = Tvec + [T[i,j]]
        T = Tvec
    return T

def round_mod(num):
    if (float(num) % 1) >= 0.5:
        x = mt.ceil(num)
    else:
        x = mt.floor(num)
    return x

def distribution(seq_str,alphabet,vectorformat):
    D = np.zeros((len(alphabet),5))
    for k in range(0,len(alphabet)):
        index = []
        for i in range(0,len(seq_str)):
            if alphabet[k]==seq_str[i]:
                index = index + [i+1]
        ntotal = len(index)
        if index!=[]:
            d1 = index[0]
            d25 = index[round_mod(0.25*ntotal)-1]
            d50 = index[round_mod(0.5*ntotal)-1]
            d75 = index[round_mod(0.75*ntotal)-1]
            d100 = index[-1]
            N = len(seq_str)
            d_vect = [d1,d25,d50,d75,d100]
            d_vect = [d/N for d in d_vect]
            D[k,:] = d_vect
    if vectorformat:
        Dvec = []
        for i in range(0,len(D[:,1])):
            for j in range(0,len(D[1,:])):
                Dvec = Dvec + [D[i,j]]
        D = Dvec
    return D

def CTD(seq_list,alphabet):
    CTD_list = []
    if type(seq_list)==str:
        n=1
    else:
        n=len(seq_list)
    for i in range(0,n):
        if type(seq_list)==str:
            seq_str = seq_list
        else:
            seq_str = seq_list.loc[i][0]
        C = composition(seq_str,alphabet)
        T = transition_pairs(seq_str,alphabet,True)
        D = distribution(seq_str,alphabet,True)
        CTDvec = C+T+D
        CTD_list.append(CTDvec)
    CTD = CTD_list
    return CTD

def dipeptidecomp(seq_str,alphabet,vectorformat):
    dipept_comp = np.zeros((len(alphabet),len(alphabet)),dtype=int)   ## rows=first aa, columns=second aa
    for i in range(0,len(seq_str)-1):
        ind_first = alphabet.find(seq_str[i])
        ind_second = alphabet.find(seq_str[i+1])
        dipept_comp[ind_first,ind_second] = dipept_comp[ind_first,ind_second] + 1
    N = len(seq_str)
    dipept_comp = dipept_comp/(N-1)
    dipeptides = dipept_comp
    if vectorformat:
        dip_vec = []
        for j in range(0,len(alphabet)):
            for i in range(0,len(alphabet)):
                dip_vec = dip_vec + [dipept_comp[i,j]]
        dipeptides = dip_vec
    return dipeptides

def tripeptidecomp(seq_str,alphabet,validtriplets,vectorformat):
    n = len(alphabet)
    tripept_comp = np.zeros((n,n,n),dtype=int)
    for i in range(0,len(seq_str)-2):
        ind_first = alphabet.find(seq_str[i])
        ind_second = alphabet.find(seq_str[i+1])
        ind_third = alphabet.find(seq_str[i+2])
        tripept_comp[ind_first,ind_second,ind_third] = tripept_comp[ind_first,ind_second,ind_third]+1
    N = len(seq_str)
    tripept_comp = tripept_comp/(N-2)
    if validtriplets!='all':
        for k in range(0,n):
            for i in range(0,n):
                for j in range(0,n):
                    if validtriplets[k,i,j]==0:
                        tripept_comp[k,i,j]=0
    tripeptides = tripept_comp
    if vectorformat:
        trip_vec = []
        for k in range(0,n):
            for i in range(0,n):
                for j in range(0,n):
                    trip_vec = trip_vec + [tripept_comp[k,i,j]]
        tripeptides = trip_vec          
    return tripeptides

def NCterminal(seq_str,alphabet,terminals,vectorformat):
    return 1

def extract_datasets_excel(dataset_directory):
    file = pd.ExcelFile(dataset_directory)
    datasets = []
    for name in file.sheet_names:
        subset = file.parse(name)
        datasets.append(subset)
    return [datasets,file.sheet_names]

#def __init__(self,dataset_path,alphabet):
    #if dataset_path.endswith('.xlsx'):
        #file = extract_datasets(dataset_path)
        #all_sheets = file[0]
        #self.description = all_sheets[0]
        #self.datasets = all_sheets[1:len(all_sheets)+1]
        #self.subsetnames = file[1][1:len(all_sheets)+1]
        #self.setname = os.path.basename(dataset_path)

In [2]:

#ohseq = onehot(seq,alfabet)
#length = []
#for i in range(0,len(seq)):
#    length = length + [len(ohseq[i])]
#print(length)
#print(len(ohseq[0]))

#df1['Secuencia'] = onehot(seq,alfabt
#print(A)
#print(onehot(seq,alfabet))
#print(df1)
A='variacion'
print(A[0:-3])

variac
